#* 3 종목, 9개 모델 학습 테스트
- 최종 선정 학습 모델: LSTM_128
- 3개 클러스터 대상  3 종목 테스트에서 가장 정확도가 높은 모델 선정
- cluster 2:  

In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
#!pip install JPype1
#!pip install konlpy
#!pip install pyMySQL
#!pip install -U finance-datareader
#!pip install attention

In [2]:
#-- Import packages --
import os
import numpy as np
import pandas as pd
#import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
#import konlpy
#from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
import itertools
import warnings
warnings.filterwarnings('ignore')
#-- Import packages :END --

In [3]:
#-- load_data_il_B() -----------------------------------------------------
# 일기준 데이터 로더: Binary 1/0 (U/D) 가져오기
def load_data_il_B(stock_code):
  # 실행확인을 위한 화면 출력
  #print(stock_code)
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트이름', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM {stock_code}_il_dataset x"
  # DB 검색결과를 dataframe에 저장
  il_data_df = pd.read_sql_query(sql, conn)
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # 레이블 'H' 는 빼기, Up/Down만 남기기
  il_data_df = il_data_df[il_data_df['label'] != 'H']
  # 레이블 숫자로 변환 return
  il_data_df['label'] = il_data_df['label'].replace({'U': 1, 'D': 0})
  # DB close
  conn.close()
  # df 반환
  return il_data_df
#-- load_data_il_B() : END--------------------------------------------

#-- load_data_il() ---------------------------------------------------
# 일기준 데이터 로더: Ternary U/H/D 가져오기
def load_data_il(stock_code):
  # 실행확인을 위한 화면 출력
  #print(stock_code)
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트이름', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM {stock_code}_il_dataset x"
  # DB 검색결과를 dataframe에 저장
  il_data_df = pd.read_sql_query(sql, conn)
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # 레이블 숫자로 변환 return
  #il_data_df['label'] = il_data_df['label'].replace({'U': 3, 'H': 2, 'D': 1})
  # DB close
  conn.close()
  # df 반환
  return il_data_df
#-- load_data_il() :End-----------------------------------------------

#-- load_data_bun_B() ------------------------------------------------
# 분기준 데이터 로더: Binary 1/0 (U/D) 가져오기
def load_data_bun_B(stock_code):
  # 실행확인을 위한 화면 출력
  #print(stock_code)
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트이름', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM {stock_code}_bun_dataset x"
  # DB 검색결과를 dataframe에 저장
  il_data_df = pd.read_sql_query(sql, conn)
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # Null 있는 행 삭제
  il_data_df = il_data_df.dropna()
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # 레이블 'H' 는 빼기, Up/Down만 남기기
  il_data_df = il_data_df[il_data_df['label'] != 'H']
  # 레이블 숫자로 변환 return
  il_data_df['label'] = il_data_df['label'].replace({'U': 1, 'D': 0})
  # DB close
  conn.close()
  # df 반환
  return il_data_df
#-- load_data_bun_B() :End--------------------------------------------

#-- load_data_bun() --------------------------------------------------
# 분기준 데이터 로더: Ternary 2/1/0(U/H/D) 가져오기
def load_data_bun(stock_code):
  # 실행확인을 위한 화면 출력
  #print(stock_code)
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트이름', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM {stock_code}_bun_dataset x"
  # DB 검색결과를 dataframe에 저장
  il_data_df = pd.read_sql_query(sql, conn)
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # Null 있는 행 삭제
  il_data_df = il_data_df.dropna()
  # 레이블 숫자로 변환 return
  il_data_df['label'] = il_data_df['label'].replace({'U': 2, 'H': 1, 'D': 0})
  # DB close
  conn.close()
  # df 반환
  return il_data_df
#-- load_data_bun() :End----------------------------------------------
#---------------------------------------------------------------------

In [4]:
# LSTM+CNN+Attention 카테고리 분류
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation \
                                  , Embedding, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from attention import Attention
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 분석대상 종목 코드
stock_code = '005930'
# data load
bun_data_df = load_data_bun(stock_code)
# 실행 확인을 위한 화면 출력
print(bun_data_df.head(3))

# data preparation: 데이터X,레이블y 분리
X = bun_data_df['text']
y = bun_data_df['label']

# 문자열 형식인 데이터X를 리스트로 변환
X= X.apply(lambda x: [int(i) for i in x.strip('[]').split()])

# data set separation: 학습셋, 테스트셋 분리
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True)

# data print, data 확인
# 레이블 카테고리 갯수 계산
category = np.max(y_train) + 1
# 실행 확인을 위한 화면 출력
print(category,'카테고리')
print(len(X_train),'학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
print(X[0])
print(y[0])

# pad_sequence 단어 수 맞춰주기
# 한 기사당 단어 갯수 제한
sequence_length = 50
X_train = sequence.pad_sequences(X_train, maxlen= sequence_length)
X_test = sequence.pad_sequences(X_test,maxlen=sequence_length)

# ** 레이블이 1/0 아닌 경우: 원핫 인코딩 처리
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# model creation
model = Sequential()
model.add(Embedding(20000,sequence_length))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=True))
model.add(Attention())
model.add(Dense(category, activation='softmax'))    # Output: ternary 이상인 경우


                                                text  label
0  [ 18 179 122 9 3749 2377 52 241 3943 1 18 415 ...      2
1  [ 743 558 11 1 908 1 12992 581 580 25 91 245 2...      2
2  [ 10 187 589 1123 6 267 29 54 2 3004 44 2459 1...      2
3 카테고리
104288 학습용 뉴스 기사
26072 테스트용 뉴스 기사
[18, 179, 122, 9, 3749, 2377, 52, 241, 3943, 1, 18, 415, 3741, 1, 179, 122]
2


In [ ]:
from tensorflow.keras.models import Model, Sequential,load_model

print('** LSTM **')
#-----------------------------------------------------------------------------------
model_name = 'LSTM_sokbo_034950_30.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTM_sokbo_034950_31.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTM_sokbo_034950_32.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
print('** LSTM_CNN **')
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNN_sokbo_034950_32.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNN_sokbo_034950_33.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNN_sokbo_034950_34.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
print('** LSTM_CNN_Attention **')
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNNATT_sokbo_034950_30.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNNATT_sokbo_034950_31.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNNATT_sokbo_034950_32.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------

print('** Transformer **')
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_034950_30.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_034950_31.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_034950_32.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------

** LSTM **
815/815 [==============================] - 7s 8ms/step - loss: 0.5656 - accuracy: 0.8007
Model: LSTM_sokbo_034950_30.hdf5, Test Accuracy: 0.8007

815/815 [==============================] - 6s 7ms/step - loss: 0.6281 - accuracy: 0.7823
Model: LSTM_sokbo_034950_31.hdf5, Test Accuracy: 0.7823

815/815 [==============================] - 7s 8ms/step - loss: 0.5796 - accuracy: 0.7941
Model: LSTM_sokbo_034950_32.hdf5, Test Accuracy: 0.7941

** LSTM_CNN **
815/815 [==============================] - 7s 8ms/step - loss: 0.5859 - accuracy: 0.7918
Model: LSTMCNN_sokbo_034950_32.hdf5, Test Accuracy: 0.7918

815/815 [==============================] - 4s 4ms/step - loss: 0.5884 - accuracy: 0.7915
Model: LSTMCNN_sokbo_034950_33.hdf5, Test Accuracy: 0.7915

815/815 [==============================] - 13s 14ms/step - loss: 0.5324 - accuracy: 0.8069
Model: LSTMCNN_sokbo_034950_34.hdf5, Test Accuracy: 0.8069

** LSTM_CNN_Attention **
815/815 [==============================] - 11s 13ms/step - los

# '005930' 종목

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GlobalAveragePooling1D, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import numpy as np

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, rate=0.1):
    attn = MultiHeadSelfAttention(embed_dim, num_heads)(inputs)
    attn = Dropout(rate)(attn)
    out = LayerNormalization(epsilon=1e-6)(inputs + attn)
    ffn = tf.keras.Sequential(
        [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
    )
    ffn_output = ffn(out)
    ffn_output = Dropout(rate)(ffn_output)
    out = LayerNormalization(epsilon=1e-6)(out + ffn_output)
    return out

# Define Transformer model
def build_transformer_model(max_seq_length, vocab_size, embed_dim, num_heads, ff_dim, num_classes):
    inputs = Input(shape=(max_seq_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    transformer_block = transformer_encoder(embedding_layer, embed_dim, num_heads, ff_dim)
    pool = GlobalAveragePooling1D()(transformer_block)
    outputs = Dense(num_classes, activation='softmax')(pool)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define hyperparameters
max_seq_length = 50
vocab_size = 20000
embed_dim = 50
num_heads = 5
ff_dim = 128
num_classes = 3
learning_rate = 1e-4

# Build and compile the model
transformer_model = build_transformer_model(max_seq_length, vocab_size, embed_dim, num_heads, ff_dim, num_classes)
optimizer = Adam(learning_rate=learning_rate)
loss_fn = CategoricalCrossentropy()
accuracy_metric = CategoricalAccuracy()

transformer_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])

# Assuming y_train is one-hot encoded
transformer_model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GlobalAveragePooling1D, Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from attention import Attention

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, rate=0.1):
    attn = MultiHeadSelfAttention(embed_dim, num_heads)(inputs)
    attn = Dropout(rate)(attn)
    out = LayerNormalization(epsilon=1e-6)(inputs + attn)
    ffn = tf.keras.Sequential(
        [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
    )
    ffn_output = ffn(out)
    ffn_output = Dropout(rate)(ffn_output)
    out = LayerNormalization(epsilon=1e-6)(out + ffn_output)
    return out

# Define Transformer model
def build_transformer_model(max_seq_length, vocab_size, embed_dim, num_heads, ff_dim, num_classes):
    inputs = Input(shape=(max_seq_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    transformer_block = transformer_encoder(embedding_layer, embed_dim, num_heads, ff_dim)
    pool = GlobalAveragePooling1D()(transformer_block)
    outputs = Dense(num_classes, activation='softmax')(pool)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define hyperparameters
max_seq_length = 50
vocab_size = 20000
embed_dim = 50
num_heads = 5
ff_dim = 128
num_classes = 3
learning_rate = 1e-4

# Build and compile the model
transformer_model = build_transformer_model(max_seq_length, vocab_size, embed_dim, num_heads, ff_dim, num_classes)
optimizer = Adam(learning_rate=learning_rate)
loss_fn = SparseCategoricalCrossentropy()
accuracy_metric = SparseCategoricalAccuracy()

transformer_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])
# Print model summary
#transformer_model.summary()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 분석대상 종목 코드
stock_code = '005930'
# data load
bun_data_df = load_data_bun(stock_code)
# 실행 확인을 위한 화면 출력
print(bun_data_df.head(3))
# data preparation: 데이터X,레이블y 분리
X = bun_data_df['text']
y = bun_data_df['label']

# 문자열 형식인 데이터X를 리스트로 변환
X= X.apply(lambda x: [int(i) for i in x.strip('[]').split()])
# data set separation: 학습셋, 테스트셋 분리
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True)
# data print, data 확인
# 레이블 카테고리 갯수 계산
category = np.max(y_train) + 1
# 실행 확인을 위한 화면 출력
print(category,'카테고리')
print(len(X_train),'학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
#print(X_train.at[0])
# pad_sequence 단어 수 맞춰주기
# 한 기사당 단어 갯수 제한
sequence_length = 50
X_train = sequence.pad_sequences(X_train, maxlen= sequence_length)
X_test = sequence.pad_sequences(X_test,maxlen=sequence_length)

# ** 레이블이 1/0 아닌 경우: 원핫 인코딩 처리
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# 조기중단 설정, early stopping callback
early_stopping_callback = EarlyStopping(
    monitor= 'val_loss'                   # 모니터 항목 : val_loss
  , patience = 10                          # patiencd    : 5 (epochs)
)


import tensorflow as tf

# 사용자 지정 레이어 등록
tf.keras.utils.get_custom_objects()['MultiHeadSelfAttention'] = MultiHeadSelfAttention

                                                text  label
0  [ 18 179 122 9 3749 2377 52 241 3943 1 18 415 ...      2
1  [ 743 558 11 1 908 1 12992 581 580 25 91 245 2...      2
2  [ 10 187 589 1123 6 267 29 54 2 3004 44 2459 1...      2
3 카테고리
104288 학습용 뉴스 기사
26072 테스트용 뉴스 기사


In [ ]:
from tensorflow.keras.models import Model, Sequential,load_model

# print('** LSTM **')
# #-----------------------------------------------------------------------------------
# model_name = 'LSTM_sokbo_005930_b32_e200_c32.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}')
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------
# #-----------------------------------------------------------------------------------
# model_name = 'LSTM_sokbo_005930_b64_e200_c32.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}')
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------
# #-----------------------------------------------------------------------------------
# model_name = 'LSTM_sokbo_005930_b128_e200_c32.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}')
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------
# print('** LSTM_CNN **')
# #-----------------------------------------------------------------------------------
# model_name = 'LSTMCNN_sokbo_005930_b32_e200_c3.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}')
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------
# #-----------------------------------------------------------------------------------
# model_name = 'LSTMCNN_sokbo_005930_b64_e200_c3.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}')
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------
# #-----------------------------------------------------------------------------------
# model_name = 'LSTMCNN_sokbo_005930_b128_e200_c3.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}')
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------
# print('** LSTM_CNN_Attention **')
# #-----------------------------------------------------------------------------------
# model_name = 'LSTMCNNATT_sokbo_005930_b32_e200_c3.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------
# #-----------------------------------------------------------------------------------
# model_name = 'LSTMCNNATT_sokbo_005930_b64_e200_c3.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------
# #-----------------------------------------------------------------------------------
# model_name = 'LSTMCNNATT_sokbo_005930_b128_e200_c3.hdf5'
# model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
# print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
# #-----------------------------------------------------------------------------------

print('** Transformer **')
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_005930_b32_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')#,custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_005930_b64_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_005930_b128_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------

** Transformer **
815/815 [==============================] - 10s 11ms/step - loss: 0.9869 - sparse_categorical_accuracy: 0.5252
Model: TFMR_sokbo_005930_b32_e200_c3.hdf5, Test Accuracy: 0.5252

815/815 [==============================] - 10s 12ms/step - loss: 1.0000 - sparse_categorical_accuracy: 0.5144
Model: TFMR_sokbo_005930_b64_e200_c3.hdf5, Test Accuracy: 0.5144

815/815 [==============================] - 10s 12ms/step - loss: 1.0221 - sparse_categorical_accuracy: 0.4921
Model: TFMR_sokbo_005930_b128_e200_c3.hdf5, Test Accuracy: 0.4921



#* 328380 종목

In [14]:
# LSTM+CNN+Attention 카테고리 분류
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation \
                                  , Embedding, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from attention import Attention
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 분석대상 종목 코드
stock_code = '328380'
# data load
bun_data_df = load_data_bun(stock_code)
# 실행 확인을 위한 화면 출력
print(bun_data_df.head(3))

# data preparation: 데이터X,레이블y 분리
X = bun_data_df['text']
y = bun_data_df['label']

# 문자열 형식인 데이터X를 리스트로 변환
X= X.apply(lambda x: [int(i) for i in x.strip('[]').split()])

# data set separation: 학습셋, 테스트셋 분리
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True)

# data print, data 확인
# 레이블 카테고리 갯수 계산
category = np.max(y_train) + 1
# 실행 확인을 위한 화면 출력
print(category,'카테고리')
print(len(X_train),'학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
print(X[0])
print(y[0])

# pad_sequence 단어 수 맞춰주기
# 한 기사당 단어 갯수 제한
sequence_length = 50
X_train = sequence.pad_sequences(X_train, maxlen= sequence_length)
X_test = sequence.pad_sequences(X_test,maxlen=sequence_length)

# ** 레이블이 1/0 아닌 경우: 원핫 인코딩 처리
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# # model creation
# model = Sequential()
# model.add(Embedding(20000,sequence_length))
# model.add(Dropout(0.5))
# model.add(LSTM(64, return_sequences=True))
# model.add(Attention())
# model.add(Dense(category, activation='softmax'))    # Output: ternary 이상인 경우


                                                text  label
0  [ 18 179 122 9 3749 2377 52 241 3943 1 18 415 ...      2
1  [ 743 558 11 1 908 1 12992 581 580 25 91 245 2...      2
2  [ 10 187 589 1123 6 267 29 54 2 3004 44 2459 1...      2
3 카테고리
104308 학습용 뉴스 기사
26078 테스트용 뉴스 기사
[18, 179, 122, 9, 3749, 2377, 52, 241, 3943, 1, 18, 415, 3741, 1, 179, 122]
2


In [15]:
from tensorflow.keras.models import Model, Sequential,load_model

print('** LSTM **')
#-----------------------------------------------------------------------------------
model_name = 'LSTM_sokbo_328380_b32_e200_c32.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTM_sokbo_328380_b64_e200_c32.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTM_sokbo_328380_b128_e200_c32.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------
print('** LSTM_CNN **')
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNN_sokbo_328380_b32_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNN_sokbo_328380_b64_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNN_sokbo_328380_b128_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------
print('** LSTM_CNN_Attention **')
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNNATT_sokbo_328380_b32_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNNATT_sokbo_328380_b64_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'LSTMCNNATT_sokbo_328380_b128_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.2f}\n')
#-----------------------------------------------------------------------------------


** LSTM **
815/815 [==============================] - 9s 10ms/step - loss: 1.0108 - accuracy: 0.5010
Model: LSTM_sokbo_328380_b32_e200_c32.hdf5, Test Accuracy: 0.50

815/815 [==============================] - 9s 10ms/step - loss: 1.0103 - accuracy: 0.4983
Model: LSTM_sokbo_328380_b64_e200_c32.hdf5, Test Accuracy: 0.50

815/815 [==============================] - 9s 10ms/step - loss: 1.0046 - accuracy: 0.5050
Model: LSTM_sokbo_328380_b128_e200_c32.hdf5, Test Accuracy: 0.50

** LSTM_CNN **
815/815 [==============================] - 5s 6ms/step - loss: 0.9521 - accuracy: 0.5568
Model: LSTMCNN_sokbo_328380_b32_e200_c3.hdf5, Test Accuracy: 0.56

815/815 [==============================] - 5s 5ms/step - loss: 0.9506 - accuracy: 0.5418
Model: LSTMCNN_sokbo_328380_b64_e200_c3.hdf5, Test Accuracy: 0.54

815/815 [==============================] - 5s 5ms/step - loss: 0.9545 - accuracy: 0.5560
Model: LSTMCNN_sokbo_328380_b128_e200_c3.hdf5, Test Accuracy: 0.56

** LSTM_CNN_Attention **
815/815 [=====

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GlobalAveragePooling1D, Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from attention import Attention

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, rate=0.1):
    attn = MultiHeadSelfAttention(embed_dim, num_heads)(inputs)
    attn = Dropout(rate)(attn)
    out = LayerNormalization(epsilon=1e-6)(inputs + attn)
    ffn = tf.keras.Sequential(
        [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
    )
    ffn_output = ffn(out)
    ffn_output = Dropout(rate)(ffn_output)
    out = LayerNormalization(epsilon=1e-6)(out + ffn_output)
    return out

# Define Transformer model
def build_transformer_model(max_seq_length, vocab_size, embed_dim, num_heads, ff_dim, num_classes):
    inputs = Input(shape=(max_seq_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    transformer_block = transformer_encoder(embedding_layer, embed_dim, num_heads, ff_dim)
    pool = GlobalAveragePooling1D()(transformer_block)
    outputs = Dense(num_classes, activation='softmax')(pool)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define hyperparameters
max_seq_length = 50
vocab_size = 20000
embed_dim = 50
num_heads = 5
ff_dim = 128
num_classes = 3
learning_rate = 1e-4

# Build and compile the model
transformer_model = build_transformer_model(max_seq_length, vocab_size, embed_dim, num_heads, ff_dim, num_classes)
optimizer = Adam(learning_rate=learning_rate)
loss_fn = SparseCategoricalCrossentropy()
accuracy_metric = SparseCategoricalAccuracy()

transformer_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])
# Print model summary
#transformer_model.summary()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 분석대상 종목 코드
stock_code = '328380'
# data load
bun_data_df = load_data_bun(stock_code)
# 실행 확인을 위한 화면 출력
print(bun_data_df.head(3))
# data preparation: 데이터X,레이블y 분리
X = bun_data_df['text']
y = bun_data_df['label']

# 문자열 형식인 데이터X를 리스트로 변환
X= X.apply(lambda x: [int(i) for i in x.strip('[]').split()])
# data set separation: 학습셋, 테스트셋 분리
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True)
# data print, data 확인
# 레이블 카테고리 갯수 계산
category = np.max(y_train) + 1
# 실행 확인을 위한 화면 출력
print(category,'카테고리')
print(len(X_train),'학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
#print(X_train.at[0])
# pad_sequence 단어 수 맞춰주기
# 한 기사당 단어 갯수 제한
sequence_length = 50
X_train = sequence.pad_sequences(X_train, maxlen= sequence_length)
X_test = sequence.pad_sequences(X_test,maxlen=sequence_length)

# ** 레이블이 1/0 아닌 경우: 원핫 인코딩 처리
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# 조기중단 설정, early stopping callback
early_stopping_callback = EarlyStopping(
    monitor= 'val_loss'                   # 모니터 항목 : val_loss
  , patience = 10                          # patiencd    : 5 (epochs)
)


import tensorflow as tf

# 사용자 지정 레이어 등록
tf.keras.utils.get_custom_objects()['MultiHeadSelfAttention'] = MultiHeadSelfAttention

                                                text  label
0  [ 18 179 122 9 3749 2377 52 241 3943 1 18 415 ...      2
1  [ 743 558 11 1 908 1 12992 581 580 25 91 245 2...      2
2  [ 10 187 589 1123 6 267 29 54 2 3004 44 2459 1...      2
3 카테고리
104308 학습용 뉴스 기사
26078 테스트용 뉴스 기사


In [13]:
print('** Transformer **')
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_328380_b32_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}')#,custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_328380_b64_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
model_name = 'TFMR_sokbo_328380_b128_e200_c3.hdf5'
model = load_model(filepath=f'./data/model/{model_name}',custom_objects={'Attention': Attention})
print(f'Model: {model_name}, Test Accuracy: {model.evaluate(X_test,y_test)[1]:.4f}\n')
#-----------------------------------------------------------------------------------

** Transformer **
815/815 [==============================] - 11s 13ms/step - loss: 0.9752 - sparse_categorical_accuracy: 0.5261
Model: TFMR_sokbo_328380_b32_e200_c3.hdf5, Test Accuracy: 0.5261

815/815 [==============================] - 11s 13ms/step - loss: 0.9839 - sparse_categorical_accuracy: 0.5172
Model: TFMR_sokbo_328380_b64_e200_c3.hdf5, Test Accuracy: 0.5172

815/815 [==============================] - 11s 13ms/step - loss: 0.9625 - sparse_categorical_accuracy: 0.5357
Model: TFMR_sokbo_328380_b128_e200_c3.hdf5, Test Accuracy: 0.5357

